# Distance Notebook
### Calculate the distance each team will need to travel over the course of the season

#### Note
- ISSUES TO ADDRESS
    - need to check the arena location data and the merge because it seems some of the locations may not be correct for example the Michigan State at Lake State is reporting a distance of double what it should 
    - The travel flag doesnt seem to be working correctly either 
- as of 9-8-24 . THe schedule table doesn't seem complete. Big Ten teams have incomplete schedule
- Does not account for nuetral site and tourny games

In [241]:
# Dependencies
import os
import sys
import pandas as pd
import numpy as np
import geopy
from geopy.distance import geodesic


In [242]:
## Paths to source data
# SCHEDULE FILE
schedule_path = os.path.join('..', 'data', 'schedule', '2024_current.csv')
schedule_data = pd.read_csv(schedule_path)
# ARENA INFO FILE
arena_path = os.path.join('..', 'data', 'arena_school_info_2.csv')
arena_data = pd.read_csv(arena_path)

# NEUTRAL SITE ARENA INFORMATION FILE
neutral_path = os.path.join('..', 'data', 'neutral_arenas_2024.csv')
neutral_arenas_df = pd.read_csv(neutral_path)

# Display data
# schedule_data.head()
# schedule_data.tail()
# arena_data.head()
neutral_arenas_df.tail()

,Flag,Arena_Name,latitude,longitude
12,"Desert Hockey Classic (at Mullett Arena, Tempe...","Mullett Arena, Tempe, Ariz.",33.426519,-111.928551
13,"at Wrigley Field, Chicago, Ill.","Wrigley Field, Chicago, Ill.",41.948343,-87.655537
14,"Connecticut Ice Tournament (at XL Center, Hart...","XL Center, Hartford, Conn.",41.768459,-72.677094
15,MVP Mayor's Cup,"MVP Arena, Albany, N.Y.",42.648594,-73.755018
16,Little Caesars,"Little Caesars Arena, Detroit, Mich",42.341110,-83.055299


## Account for nuetral site games

In [243]:
# Identify rows in the schedule table that involve neutral site games
# by checking if the 'Conference' or 'Game_Notes' contains a match with the 'Flag' in the neutral arenas table.

# Helper function to find if any flag appears in the Conference or Game_Notes columns
def is_neutral_game(row, flags):
    # Check for flags in both Conference and Game_Notes, ensuring correct handling of NaN
    conference_match = any(flag in str(row['Conference']) for flag in flags)
    notes_match = any(flag in str(row['Game_Notes']) for flag in flags)
    return conference_match or notes_match

# Extract the list of flags from the neutral arenas table (assuming already loaded neutral_arenas_df)
neutral_flags = neutral_arenas_df['Flag'].tolist()

# Apply the function to the schedule data to identify neutral site games
schedule_data['Is_Neutral_Game'] = schedule_data.apply(is_neutral_game, axis=1, flags=neutral_flags)

# Filter the schedule for neutral site games
neutral_site_games = schedule_data[schedule_data['Is_Neutral_Game']]

# # Remove Rows that have TBD or a / in one of the team columns
# neutral_site_games = neutral_site_games[~neutral_site_games['Home_Team'].str.contains('/')]
# neutral_site_games = neutral_site_games[~neutral_site_games['Away_Team'].str.contains('/')]
# neutral_site_games = neutral_site_games[~neutral_site_games['Home_Team'].str.contains('TBD')]
# neutral_site_games = neutral_site_games[~neutral_site_games['Away_Team'].str.contains('TBD')]


# Display the neutral site games and info
# neutral_site_games.head()
# neutral_site_games.tail()
# # neutral_site_games.info()
# # neutral_flags
# # Reindex the neutral site games
# neutral_site_games.reset_index(drop=True, inplace=True)
# neutral_site_games

### Calculate the distance to neutral site locations

In [244]:
### refactor the code to include the distance between the two teams


# Helper function to calculate the distance between two points (lat, lon)
def calculate_distance(lat1, lon1, lat2, lon2):
    if pd.notnull(lat1) and pd.notnull(lon1) and pd.notnull(lat2) and pd.notnull(lon2):
        return geodesic((lat1, lon1), (lat2, lon2)).miles
    else:
        return None  # Return None if any coordinates are missing

# Function to merge team location data and neutral arenas
def merge_team_and_arena_data(schedule_df, team_df, arena_df):
    # Merge team locations (away and home teams)
    schedule_df = schedule_df.merge(
        team_df[['Team', 'Latitude', 'Longitude']], 
        left_on='Away_Team', right_on='Team', how='left', suffixes=('', '_away')
    )
    schedule_df = schedule_df.merge(
        team_df[['Team', 'Latitude', 'Longitude']], 
        left_on='Home_Team', right_on='Team', how='left', suffixes=('', '_home')
    )

    # Merge on 'Conference' column first
    schedule_df = schedule_df.merge(
        arena_df[['Flag', 'latitude', 'longitude']], 
        left_on='Conference', right_on='Flag', how='left'
    )

    # Handle rows where the Conference merge did not work by checking 'Game_Notes'
    missing_coords_df = schedule_df[schedule_df['latitude'].isnull()].copy()

    def match_flag_in_game_notes(row, arena_df):
        for _, flag_row in arena_df.iterrows():
            if flag_row['Flag'] in str(row['Game_Notes']):
                return flag_row['latitude'], flag_row['longitude']
        return None, None

    # Apply partial matching function for missing coordinates
    missing_coords_df[['latitude', 'longitude']] = missing_coords_df.apply(
        lambda row: match_flag_in_game_notes(row, arena_df), axis=1, result_type="expand"
    )

    # Fill missing latitude/longitude
    schedule_df.loc[schedule_df['latitude'].isnull(), ['latitude', 'longitude']] = missing_coords_df[['latitude', 'longitude']]

    return schedule_df

# # Function to calculate distances to neutral site
# def calculate_team_distances(schedule_df):
#     schedule_df['Away_Distance'] = schedule_df.apply(
#         lambda row: calculate_distance(row['Latitude'], row['Longitude'], row['latitude'], row['longitude']), axis=1
#     )
#     schedule_df['Home_Distance'] = schedule_df.apply(
#         lambda row: calculate_distance(row['Latitude_home'], row['Longitude_home'], row['latitude'], row['longitude']), axis=1
#     )

#     return schedule_df

# # Refactor into steps
# neutral_site_games = merge_team_and_arena_data(schedule_data, team_locations_df, neutral_arenas_df)
# neutral_site_games = calculate_team_distances(neutral_site_games)

# Filter out rows with missing distances
# neutral_site_games = neutral_site_games.dropna(subset=['Away_Distance', 'Home_Distance'])

# Output results
output_path = '../TEMP/neutral_site_games_distances_TEST2.csv'
neutral_site_games.to_csv(output_path, index=False)

## PRINT DF INFOR BEFORE THE CLEANING
# print(neutral_site_games.info())

In [245]:
### CLEANING BASED ON LOOK AT OUTPUT FROM CELL ABOVE


# Remove Rows that have TBD or a / in one of the team columns
neutral_site_games = neutral_site_games[~neutral_site_games['Home_Team'].str.contains('/')]
neutral_site_games = neutral_site_games[~neutral_site_games['Away_Team'].str.contains('/')]
neutral_site_games = neutral_site_games[~neutral_site_games['Home_Team'].str.contains('TBD')]
neutral_site_games = neutral_site_games[~neutral_site_games['Away_Team'].str.contains('TBD')]

# Print DF INFO AFTER CLEANING
# print(neutral_site_games.info())

# OUTPUT CSV FOR CHECKING INTO TEMP FOLDER
output_path = '../TEMP/neutral_site_games_distances_CLEANED2.csv'
neutral_site_games.to_csv(output_path, index=False)

## NOTE - STOPING HERE 
- above is looking OK cleaning / filtering the nuetral games. leaving 30 at the moment (Sunday before watching lions game) the 

In [246]:
neutral_site_games.head()

,Date,Conference,Game_Notes,Away_Team,Away_Team_Link,Away_Score,Home_Team,Home_Team_Link,Home_Score,OT,Box_Link,Metrics_Link,Day,Game_ID,Is_Neutral_Game
46,2024-10-11,"Ice Breaker (at Orleans Arena, Paradise, Nev.)",NaN,Omaha,/reports/team/Omaha/37,NaN,Massachusetts,/reports/team/Massachusetts/27,NaN,NaN,NaN,NaN,Friday,2024-10-11_Massachusetts_Omaha,True
47,2024-10-11,"Ice Breaker (at Orleans Arena, Paradise, Nev.)",NaN,Minnesota,/reports/team/Minnesota/34,NaN,Air Force,/reports/team/Air-Force/1,NaN,NaN,NaN,NaN,Friday,2024-10-11_Air Force_Minnesota,True
89,2024-10-12,Non-Conference,US Hockey Hall of Fame game,Providence,/reports/team/Providence/46,NaN,North Dakota,/reports/team/North-Dakota/40,NaN,NaN,NaN,NaN,Saturday,2024-10-12_North Dakota_Providence,True
202,2024-10-26,Non-Conference,"at Xcel Energy Center, St. Paul, Minn.",Minnesota,/reports/team/Minnesota/34,NaN,St. Thomas,/reports/team/St-Thomas/63,NaN,NaN,NaN,NaN,Saturday,2024-10-26_St. Thomas_Minnesota,True
366,2024-11-16,CCHA,"at Centre Ice Arena, Traverse City, Mich.",Ferris State,/reports/team/Ferris-State/21,NaN,Lake Superior,/reports/team/Lake-Superior/24,NaN,NaN,NaN,NaN,Saturday,2024-11-16_Lake Superior_Ferris State,True


In [247]:
## New Approach - create a new table and structure for the neutral site games - add them to the agg count at the end

## Assign a location to each game - based on the FLag column from neutral_arenas_df

# Drop the columns that are not needed
neutral_site_games = neutral_site_games.drop(columns=['Away_Team_Link', 'Away_Score', 'Home_Team_Link', 'Home_Score', 'OT', 'Box_Link', 'Metrics_Link'])

# Reinex the DF
neutral_site_games.reset_index(drop=True, inplace=True)

# If Game_Notes is NaN fill with Conference
neutral_site_games['Game_Notes'] = neutral_site_games['Game_Notes'].fillna(neutral_site_games['Conference'])

# Assign a location to each game
# Look for Game_Notes that contain the Flag from neutral_arenas_df
# If there is a match, assign the location to the game

# Helper function to assign latitude and longitude to each game
def assign_location(row, arena_df):
    for _, flag_row in arena_df.iterrows():
        if flag_row['Flag'] in str(row['Game_Notes']):
            return flag_row['latitude'], flag_row['longitude']
    return None, None

# Extract the list of flags from the neutral arenas table (assuming already loaded neutral_arenas_df)
neutral_flags = neutral_arenas_df['Flag'].tolist()

# Apply the function to the schedule data to identify neutral site games
neutral_site_games[['latitude', 'longitude']] = neutral_site_games.apply(assign_location, axis=1, arena_df=neutral_arenas_df, result_type="expand")






# neutral_site_games.head()
# neutral_site_games.tail()
# neutral_site_games




## WORKING ABOVE

In [248]:
### Calculate the distance between the two teams for each game

# Helper function to calculate the distance between two points (lat, lon)
def calculate_distance(lat1, lon1, lat2, lon2):
    if pd.notnull(lat1) and pd.notnull(lon1) and pd.notnull(lat2) and pd.notnull(lon2):
        return geodesic((lat1, lon1), (lat2, lon2)).miles
    else:
        return None  # Return None if any coordinates are missing

# Function to calculate distances to neutral site
def calculate_team_distances(schedule_df):
    schedule_df['Away_Distance'] = schedule_df.apply(
        lambda row: calculate_distance(row['Latitude'], row['Longitude'], row['latitude'], row['longitude']), axis=1
    )
    schedule_df['Home_Distance'] = schedule_df.apply(
        lambda row: calculate_distance(row['Latitude_home'], row['Longitude_home'], row['latitude'], row['longitude']), axis=1
    )

    return schedule_df

# Refactor into steps
neutral_site_games = merge_team_and_arena_data(schedule_data, arena_data, neutral_arenas_df)
neutral_site_games = calculate_team_distances(neutral_site_games)

# Filter out rows with missing distances
neutral_site_games = neutral_site_games.dropna(subset=['Away_Distance', 'Home_Distance'])


# Output results
output_path = '../TEMP/neutral_site_games_distances_TESTv3.csv'
neutral_site_games.to_csv(output_path, index=False)

neutral_site_games.head()
neutral_site_games.tail()

,Date,Conference,Game_Notes,Away_Team,Away_Team_Link,Away_Score,Home_Team,Home_Team_Link,Home_Score,OT,...,Latitude,Longitude,Team_home,Latitude_home,Longitude_home,Flag,latitude,longitude,Away_Distance,Home_Distance
619,2025-01-03,Big Ten,"at Wrigley Field, Chicago, Ill.",Notre Dame,/reports/team/Notre-Dame/43,NaN,Penn State,/reports/team/Penn-State/60,NaN,NaN,...,41.693775,-86.230831,Penn State,40.806567,-77.857010,NaN,41.948343,-87.655537,75.618776,515.088064
648,2025-01-04,Big Ten,"at Wrigley Field, Chicago, Ill.",Michigan State,/reports/team/Michigan-State/32,NaN,Wisconsin,/reports/team/Wisconsin/58,NaN,NaN,...,42.728266,-84.489366,Wisconsin,43.069489,-89.396972,NaN,41.948343,-87.655537,170.821767,117.883420
793,2025-01-24,"Connecticut Ice Tournament (at XL Center, Hart...",NaN,Quinnipiac,/reports/team/Quinnipiac/47,NaN,Connecticut,/reports/team/Connecticut/17,NaN,NaN,...,41.421634,-72.889213,Connecticut,41.379757,-72.104343,"Connecticut Ice Tournament (at XL Center, Hart...",41.768459,-72.677094,26.337441,40.007306
795,2025-01-24,"Connecticut Ice Tournament (at XL Center, Hart...",NaN,Yale,/reports/team/Yale/59,NaN,Sacred Heart,/reports/team/Sacred-Heart/51,NaN,NaN,...,41.316789,-72.925016,Sacred Heart,42.580531,-83.225961,"Connecticut Ice Tournament (at XL Center, Hart...",41.768459,-72.677094,33.717142,544.114229
939,2025-02-08,Big Ten,"at Little Caesars Arena, Detroit, Mich.",Michigan State,/reports/team/Michigan-State/32,NaN,Michigan,/reports/team/Michigan/31,NaN,NaN,...,42.728266,-84.489366,Michigan,42.254222,-83.778199,NaN,42.341110,-83.055299,77.929746,37.523898


In [258]:
### DESIRED OUTPUT
# Date, Game_ID, Game_Notes, Team, Distance for each team in each game

# Create a new DataFrame to store the results
neutral_site_games_agg = pd.DataFrame(columns=['Date', 'Game_ID', 'Game_Notes', 'Team', 'Distance'])

# Iterate through the neutral_site_games DataFrame and add the data to the new DataFrame
rows = []  # Use a list to accumulate rows for better performance
for index, row in neutral_site_games.iterrows():
    # Add the Away Team data
    rows.append({
        'Date': row['Date'],
        'Game_ID': row['Game_ID'],
        'Game_Notes': row['Game_Notes'],
        'Team': row['Away_Team'],
        'Distance': row['Away_Distance']
    })
    
    # Add the Home Team data
    rows.append({
        'Date': row['Date'],
        'Game_ID': row['Game_ID'],
        'Game_Notes': row['Game_Notes'],
        'Team': row['Home_Team'],
        'Distance': row['Home_Distance']
    })

# Convert the list of rows into a DataFrame and concatenate
neutral_site_games_agg = pd.concat([neutral_site_games_agg, pd.DataFrame(rows)], ignore_index=True)

# Output results
output_path = '../TEMP/neutral_site_games_distances_agg_TESTv4.csv'
neutral_site_games_agg.to_csv(output_path, index=False)

# Display the first and last few rows of the DataFrame
neutral_site_games_agg.head()
neutral_site_games_agg.tail()

C:\Users\jbanc\AppData\Local\Temp\ipykernel_9824\2686225299.py:29: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  neutral_site_games_agg = pd.concat([neutral_site_games_agg, pd.DataFrame(rows)], ignore_index=True)


,Date,Game_ID,Game_Notes,Team,Distance
53,2025-01-24,2025-01-24_Connecticut_Quinnipiac,NaN,Connecticut,40.007306
54,2025-01-24,2025-01-24_Sacred Heart_Yale,NaN,Yale,33.717142
55,2025-01-24,2025-01-24_Sacred Heart_Yale,NaN,Sacred Heart,544.114229
56,2025-02-08,2025-02-08_Michigan_Michigan State,"at Little Caesars Arena, Detroit, Mich.",Michigan State,77.929746
57,2025-02-08,2025-02-08_Michigan_Michigan State,"at Little Caesars Arena, Detroit, Mich.",Michigan,37.523898


## Continued on Sunday Above

In [250]:
#### ORIGINAL - KIND OF A MESS

# from geopy.distance import geodesic
# # Create a copy of the team location data
# team_locations_df = arena_data.copy()

# # Helper function to calculate the distance between two points (lat, lon)
# def calculate_distance(lat1, lon1, lat2, lon2):
#     # Ensure none of the coordinates are missing (NaN)
#     if pd.notnull(lat1) and pd.notnull(lon1) and pd.notnull(lat2) and pd.notnull(lon2):
#         return geodesic((lat1, lon1), (lat2, lon2)).miles
#     else:
#         return None  # Return None if any coordinates are missing


# # Step 1: Merge on 'Conference' column
# neutral_site_games = neutral_site_games.merge(
#     team_locations_df[['Team', 'Latitude', 'Longitude']], 
#     left_on='Away_Team', right_on='Team', how='left', suffixes=('', '_away')
# )
# neutral_site_games = neutral_site_games.merge(
#     team_locations_df[['Team', 'Latitude', 'Longitude']], 
#     left_on='Home_Team', right_on='Team', how='left', suffixes=('', '_home')
# )
# neutral_site_games = neutral_site_games.merge(
#     neutral_arenas_df[['Flag', 'latitude', 'longitude']], 
#     left_on='Conference', right_on='Flag', how='left'
# )

# # Step 2: Identify rows with missing latitude and longitude (no match found in Conference)
# missing_neutral_arenas = neutral_site_games[neutral_site_games['latitude'].isnull()]


# ### APROACH 2
# # Update the merge function to handle partial matching of 'Flag' inside 'Game_Notes'
# def merge_neutral_sites_partial_match(schedule_df, arenas_df):
#     # Step 1: First merge on 'Conference' column
#     merged_df = schedule_df.merge(arenas_df[['Flag', 'latitude', 'longitude']], 
#                                   left_on='Conference', right_on='Flag', how='left')

#     # Step 2: Identify rows with missing latitude and longitude
#     missing_coords_df = merged_df[merged_df['latitude'].isnull()].copy()

#     # Step 3: Define a function to find if any Flag from arenas_df is in the Game_Notes
#     def match_flag_in_game_notes(row, flags_df):
#         for _, flag_row in flags_df.iterrows():
#             if flag_row['Flag'] in str(row['Game_Notes']):
#                 return flag_row['latitude'], flag_row['longitude']
#         return None, None

#     # Apply the function to rows with missing coordinates to perform partial matching
#     missing_coords_df[['latitude', 'longitude']] = missing_coords_df.apply(
#         lambda row: match_flag_in_game_notes(row, arenas_df), axis=1, result_type="expand"
#     )

#     # Step 4: Update the original merged_df with the new latitude/longitude values
#     merged_df.loc[merged_df['latitude'].isnull(), ['latitude', 'longitude']] = missing_coords_df[['latitude', 'longitude']]

#     return merged_df

# # Apply the updated function to the full schedule
# neutral_site_games_partial_match = merge_neutral_sites_partial_match(schedule_data, neutral_arenas_df)

# # # Step 3: Merge on 'Game_Notes' column to update missing rows
# # missing_neutral_arenas = missing_neutral_arenas.merge(
# #     neutral_arenas_df[['Flag', 'latitude', 'longitude']], 
# #     left_on='Game_Notes', right_on='Flag', how='left', suffixes=('_conf', '_notes')
# # )

# # Step 4: Fill missing 'latitude' and 'longitude' with values from 'Game_Notes' match
# neutral_site_games.loc[neutral_site_games['latitude'].isnull(), 'latitude'] = missing_neutral_arenas['latitude_notes']
# neutral_site_games.loc[neutral_site_games['longitude'].isnull(), 'longitude'] = missing_neutral_arenas['longitude_notes']
# # # Merge the schedule data for neutral site games with the team location data (for both home and away teams)
# # neutral_site_games = neutral_site_games.merge(team_locations_df[['Team', 'Latitude', 'Longitude']], 
# #                                               left_on='Away_Team', right_on='Team', how='left', suffixes=('', '_away'))
# # neutral_site_games = neutral_site_games.merge(team_locations_df[['Team', 'Latitude', 'Longitude']], 
# #                                               left_on='Home_Team', right_on='Team', how='left', suffixes=('', '_home'))

# # # Merge the neutral site games with the neutral arena coordinates
# # neutral_site_games = neutral_site_games.merge(neutral_arenas_df[['Flag', 'latitude', 'longitude']], 
# #                                               left_on='Conference', right_on='Flag', how='left')

# # # Make sure to check the Game_Notes column if the Conference column did not match
# # missing_neutral_arenas = neutral_site_games[neutral_site_games['latitude'].isnull()]
# # missing_neutral_arenas = missing_neutral_arenas.merge(neutral_arenas_df[['Flag', 'latitude', 'longitude']],
# #                                                         left_on='Game_Notes', right_on='Flag', how='left')



# # Calculate the distance for away and home teams to the neutral site
# neutral_site_games['Away_Distance'] = neutral_site_games.apply(
#     lambda row: calculate_distance(row['Latitude'], row['Longitude'], row['latitude'], row['longitude']), axis=1)
# neutral_site_games['Home_Distance'] = neutral_site_games.apply(
#     lambda row: calculate_distance(row['Latitude_home'], row['Longitude_home'], row['latitude'], row['longitude']), axis=1)

# # Filter out rows with missing distance information
# # neutral_site_games = neutral_site_games.dropna(subset=['Away_Distance', 'Home_Distance'])

# # Display the neutral site games with distance information
# neutral_site_games[['Date', 'Away_Team', 'Home_Team', 'Away_Distance', 'Home_Distance']]
# # neutral_site_games.head(25)
# # neutral_site_games.tail(15)
# # neutral_site_games

# # OUTPUT TO TEMP FILE FOR TESTING
# output_path = os.path.join('..', 'TEMP', 'neutral_site_games_distances_TEST1.csv')
# neutral_site_games.to_csv(output_path, index=False)

In [251]:
### CURRENTLY UNUSED CODE
# ## List of text in Conference or Game_Notes columns that indicate a neutral site game
# nuetral_site_flags = ('Ice Breaker (at Orleans Arena, Paradise, Nev.)',
#     'at Xcel Energy Center, St. Paul, Minn.','at Centre Ice Arena, Traverse City, Mich.',
#     'Friendship Four (at SSE Arena, Belfast, N. Ireland)','Adirondack Winter Invitational (at 1980 Rink Herb Brooks Arena, Lake Placid, N.Y.)',
#     'at Madison Square Garden, New York','Ledyard Bank Classic (at Thompson Arena, Hanover, N.H.)','Holiday Face-Off (at Fiserv Forum, Milwaukee, Wis.)',
#     'Great Lakes Invitational (at Van Andel Arena, Grand Rapids, Mich.)',
#     'at Acrisure Arena, Palm Springs, Calif.','Desert Hockey Classic (at Mullett Arena, Tempe, Ariz.)',
#     'at Wrigley Field, Chicago, Ill.','Connecticut Ice Tournament (at XL Center, Hartford, Conn.)',"Mayor's Cup",
#     'at Little Caesars Arena, Detroit, Mich.',"Capital Hockey Classic â€“ Capital One Arena, Washington, D.C.","US Hockey Hall of Fame game"
#     )

### Merge the arena info into schedule table

In [252]:
# Merge the schedule data with the arena data to include home and away team locations

# First, ensure team names match between datasets
# We will merge on the 'Team' column in the arena data and 'Home_Team'/'Away_Team' in the schedule data
merged_data = schedule_data.merge(arena_data[['Team', 'Latitude', 'Longitude']], left_on='Home_Team', right_on='Team', how='left')
merged_data = merged_data.rename(columns={'Latitude': 'Home_Latitude', 'Longitude': 'Home_Longitude'})

# Merge again for the away teams
merged_data = merged_data.merge(arena_data[['Team', 'Latitude', 'Longitude']], left_on='Away_Team', right_on='Team', how='left')
merged_data = merged_data.rename(columns={'Latitude': 'Away_Latitude', 'Longitude': 'Away_Longitude'})

# Drop the unnecessary 'Team' columns from the merged data
merged_data = merged_data.drop(columns=['Team_x', 'Team_y'])

# Display the first few rows of the merged data to verify the result
merged_data.head()

,Date,Conference,Game_Notes,Away_Team,Away_Team_Link,Away_Score,Home_Team,Home_Team_Link,Home_Score,OT,Box_Link,Metrics_Link,Day,Game_ID,Is_Neutral_Game,Home_Latitude,Home_Longitude,Away_Latitude,Away_Longitude
0,2024-10-04,Non-Conference,NaN,Michigan State,/reports/team/Michigan-State/32,NaN,Lake Superior,/reports/team/Lake-Superior/24,NaN,NaN,NaN,NaN,Friday,2024-10-04_Lake Superior_Michigan State,False,46.784622,-92.144841,42.728266,-84.489366
1,2024-10-04,Non-Conference,NaN,Minnesota State,/reports/team/Minnesota-State/35,NaN,Michigan,/reports/team/Michigan/31,NaN,NaN,NaN,NaN,Friday,2024-10-04_Michigan_Minnesota State,False,42.254222,-83.778199,44.144307,-93.995043
2,2024-10-04,Non-Conference,NaN,Arizona State,/reports/team/Arizona-State/61,NaN,Air Force,/reports/team/Air-Force/1,NaN,NaN,NaN,NaN,Friday,2024-10-04_Air Force_Arizona State,False,39.013739,-104.883727,33.447156,-111.910867
3,2024-10-05,Non-Conference,NaN,Providence,/reports/team/Providence/46,NaN,Union,/reports/team/Union/54,NaN,NaN,NaN,NaN,Saturday,2024-10-05_Union_Providence,False,42.818004,-73.924824,41.844005,-71.434748
4,2024-10-05,Non-Conference,NaN,Holy Cross,/reports/team/Holy-Cross/23,NaN,Boston University,/reports/team/Boston-University/10,NaN,NaN,NaN,NaN,Saturday,2024-10-05_Boston University_Holy Cross,False,42.353838,-71.120653,42.239239,-71.807961


### Caculate the distance between each school
- Using Haversine equation to calculate the straight line distance between two sets of lat/lon coodinates

In [253]:
# Function to calculate the haversine distance (numpy) between two points (lat1, lon1) and (lat2, lon2)
def haversine(lat1, lon1, lat2, lon2):
    # Convert degrees to radians
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])

    # Haversine formula
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = np.sin(dlat / 2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2)**2
    c = 2 * np.arcsin(np.sqrt(a))

    # Radius of Earth in miles
    r = 3956
    return c * r

# Apply the Haversine function to each row to calculate the distance between the home and away arenas
merged_data['Distance_Miles'] = merged_data.apply(
    lambda row: haversine(row['Home_Latitude'], row['Home_Longitude'], row['Away_Latitude'], row['Away_Longitude']),
    axis=1
)

# Display the updated data with the calculated distance
merged_data[['Home_Team', 'Away_Team', 'Distance_Miles']].head()

,Home_Team,Away_Team,Distance_Miles
0,Lake Superior,Michigan State,467.960246
1,Michigan,Minnesota State,530.146744
2,Air Force,Arizona State,548.193384
3,Union,Providence,143.786679
4,Boston University,Holy Cross,35.982031


### Filter out results to avoid double counting games on weekend series

- Travel_Flag to account for consecutive games played at the same venue within a 3-day span. If a team plays multiple games at the same venue within this period, travel is only counted for the first game.

- The Adjusted_Travel_Distance column reflects the distance a team will travel for each game, considering the consecutive game rule.

In [254]:
### VERSION 2
# Update logic to handle non-consecutive rows by grouping first

# Convert 'Date' column to datetime format for easier manipulation
merged_data['Date'] = pd.to_datetime(merged_data['Date'])

# Reset the travel flag
merged_data['Travel_Flag'] = 1

# Sort the data by 'Away_Team', 'Home_Team', and 'Date' to ensure games are grouped correctly
merged_data = merged_data.sort_values(by=['Away_Team', 'Home_Team', 'Date'])

# Group by 'Away_Team' and 'Home_Team', then iterate through each group to set the travel flag
for (away_team, home_team), group in merged_data.groupby(['Away_Team', 'Home_Team']):
    group = group.sort_values(by='Date')  # Sort by date within each group
    
    # Iterate through the group to check for consecutive games
    for i in range(1, len(group)):
        current_game = group.iloc[i]
        previous_game = group.iloc[i - 1]
        
        # Check if the games are within 3 days
        if (current_game['Date'] - previous_game['Date']).days <= 3:
            # Set the travel flag to 0 for the current game
            merged_data.loc[current_game.name, 'Travel_Flag'] = 0

# Only consider rows where travel flag is 1 for calculating total travel distance
merged_data['Adjusted_Travel_Distance'] = merged_data['Distance_Miles'] * merged_data['Travel_Flag']

# Display the updated data with the travel flag and adjusted distance
merged_data[['Away_Team', 'Home_Team', 'Date', 'Distance_Miles', 'Travel_Flag', 'Adjusted_Travel_Distance']].head()

# OUTPUT TABLE TO TEMP FILE FOR TESTING
output_path = os.path.join('..', 'TEMP', 'distance_test_v2.csv')
merged_data.to_csv(output_path, index=False)

In [255]:
### ORIGINAL CODE FOR TRAVEL DISTANCE CALCULATION

# # Convert 'Date' column to datetime format for easier manipulation
# merged_data['Date'] = pd.to_datetime(merged_data['Date'])

# # Sort the data by 'Away_Team' and 'Date' to identify consecutive games at the same venue
# merged_data = merged_data.sort_values(by=['Away_Team', 'Date'])

# # Initialize a flag column to indicate whether the travel should be counted (1 = yes, 0 = no)
# merged_data['Travel_Flag'] = 1

# # Iterate through the rows and check for consecutive games at the same venue within a 3-day span
# for i in range(1, len(merged_data)):
#     current_game = merged_data.iloc[i]
#     previous_game = merged_data.iloc[i - 1]
    
#     # Check if the away team is the same, and the venue (home team) is the same, and the games are within 3 days
#     if (current_game['Away_Team'] == previous_game['Away_Team'] and
#         current_game['Home_Team'] == previous_game['Home_Team'] and
#         (current_game['Date'] - previous_game['Date']).days <= 3):
#         # Set the travel flag to 0 for the current game (no additional travel)
#         merged_data.at[i, 'Travel_Flag'] = 0

# # Only consider rows where travel flag is 1 for calculating total travel distance
# merged_data['Adjusted_Travel_Distance'] = merged_data['Distance_Miles'] * merged_data['Travel_Flag']

# # Display the updated data with the travel flag and adjusted distance
# merged_data[['Away_Team', 'Home_Team', 'Date', 'Distance_Miles', 'Travel_Flag', 'Adjusted_Travel_Distance']].head()


## Aggregate Total Travel Distance for Each Team AND
## Calculate and store the Trip Count and the Average trip distance

In [256]:
# Only consider rows where travel flag is 1 for calculating total travel distance
merged_data['Adjusted_Travel_Distance'] = merged_data['Distance_Miles'] * merged_data['Travel_Flag']

# Calculate the total travel distance per team
team_travel_distances = merged_data.groupby('Away_Team')['Adjusted_Travel_Distance'].sum().reset_index()
team_travel_distances.columns = ['Team', 'Total_Travel_Distance']

# Step 4: Adding Trip Count and Average Trip Distance

# Calculate the number of trips for each team
trip_count = merged_data[merged_data['Travel_Flag'] == 1].groupby('Away_Team').size().reset_index(name='Trip_Count')

# Merge trip count with travel distances
team_travel_data = pd.merge(team_travel_distances, trip_count, left_on='Team', right_on='Away_Team', how='left').drop(columns='Away_Team')

# Calculate average trip distance
team_travel_data['Average_Trip_Distance'] = team_travel_data['Total_Travel_Distance'] / team_travel_data['Trip_Count']

# Display the top 5 teams with the highest average trip distance
team_travel_data = team_travel_data.sort_values(by='Average_Trip_Distance', ascending=False)
team_travel_data.head()

,Team,Total_Travel_Distance,Trip_Count,Average_Trip_Distance
2,Alaska-Anchorage,30663.509981,12,2555.292498
1,Alaska,28413.140021,12,2367.761668
4,Arizona State,10730.958743,9,1192.328749
0,Air Force,10221.145428,9,1135.682825
25,Denver,10233.783848,10,1023.378385


## Output the aggrigate as a csv file (Version 1)
- does not account for nuetral site games, mostly games in tournaments. These games often have teams listed like this 'WMU/Michigan State' or 'Bc/BU' for example

In [257]:
# Output into the TEMP folder
output_path = os.path.join('..', 'TEMP', 'team_travel_distances_v3.csv')
team_travel_data.to_csv(output_path, index=False)
print(f"Output saved to: {output_path}")

Output saved to: ..\TEMP\team_travel_distances_v3.csv
